<a href="https://colab.research.google.com/github/darshbaxi/Tensorflow-Practice/blob/main/NLP_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import keras

In [ ]:
df=pd.read_csv('SMSSpamCollection.txt',sep='\t',names=["label","message"])

In [ ]:
df.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
import re


In [ ]:
import nltk

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
final=[]
for i in range(len(df)):
  review=re.sub('[^a-zA-Z]',' ',df['message'][i])
  review=review.lower()
  review=review.split()
  review=[ps.stem(word) for word in review if word not in set(stopwords.words('english'))]
  review=' '.join(review)
  final.append(review)

In [ ]:
final

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(binary=True)
x=cv.fit_transform(final).toarray()

In [ ]:
y=df['label']


In [ ]:
y

0        ham
1        ham
2       spam
3        ham
4        ham
        ... 
5567    spam
5568     ham
5569     ham
5570     ham
5571     ham
Name: label, Length: 5572, dtype: object

In [ ]:
y=pd.get_dummies(y)

In [ ]:
y=y.drop(['spam'],axis=1)

In [ ]:
x.shape

(5572, 6296)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(force_alpha=True)
clf.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB(force_alpha=True)

In [ ]:
y_pred=clf.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred,y_test)
score

0.977705274605764

In [ ]:
from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(y_pred,y_test)
matrix

array([[ 235,   30],
       [  11, 1563]])

#WORD2VEC

In [ ]:
from nltk.stem import WordNetLemmatizer as wl
lammtizer=wl()
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:

final=[]
for i in range(len(df)):
  review=re.sub('[^a-zA-Z]',' ',df['message'][i])
  review=review.lower()
  review=review.split()
  review=[lammtizer.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
  review=' '.join(review)
  final.append(review)

In [ ]:
from nltk import sent_tokenize
from gensim.utils import simple_preprocess

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
words=[]
for sent in final:
    sent_token=sent_tokenize(sent)
    for sent in sent_token:
        words.append(simple_preprocess(sent))

In [ ]:
words

In [ ]:
import gensim
model=gensim.models.Word2Vec(words,window=5)

In [ ]:
model

In [ ]:
def avg_word2vec(doc):
    # remove out-of-vocabulary words
    #sent = [word for word in doc if word in model.wv.index_to_key]
    #print(sent)

    return np.mean([model.wv[word] for word in doc if word in model.wv.index_to_key],axis=0)
                #or [np.zeros(len(model.wv.index_to_key))], axis=0)


In [ ]:
X=[]
for i in range(len(words)):
    X.append(avg_word2vec(words[i]))


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
